## 07 역전파 자동화

### colab 접속

In [1]:
if __name__ == '__main__':
  !pip install import_ipynb
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/밑바닥부터 시작하는 딥러닝3

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp37-none-any.whl size=2976 sha256=b60a27977e23c8cd3f53f265bf3ae01124ff07e0f6e265180fb821fbaf4de622
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
Mounted at /content/drive
/content/drive/MyDrive/밑바닥부터 시작하는 딥러닝3


### 7.1 역전파 자동화의 시작
+ 동적 계산의 시작

In [ ]:
# class Variable:
#   def __init__(self, data) :
#     self.data = data
#     self.grad = None

In [9]:
# creator 인스턴스 변수 추가
class Variable:
  def __init__(self, data) :
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func

In [ ]:
# class Function:
#   def __call__(self, input):
#     x = input.data
#     y= self.forward(x)
#     output = Variable(y)
#     self.input = input
#     return output
  
#   def forward(self,x):
#     raise NotImplementedError()

#   def backward(self,gy):
#     raise NotImplementedError()

In [10]:
class Function:
  def __call__(self, input):
    x = input.data
    y= self.forward(x)
    output = Variable(y)
    output.set_creator(self)
    self.input = input
    self.output = output
    return output
  
  def forward(self,x):
    raise NotImplementedError()

  def backward(self,gy):
    raise NotImplementedError()

In [11]:
class Square(Function):
  def forward(self, x):
    y = x**2
    return y
  def backward(self, gy):
    x = self.input.data
    gx = 2*x*gy
    return gx

In [12]:
class Exp(Function):
  def forward(self, x):
    y = np.exp(x)
    return y
  
  def backward(self, gy):
    x = self.input.data
    gx = np.exp(x)*gy
    return gx

#### creator
+ 변수는 결국 함수를 통해서 만들어진다. creator는 변수가 어디서 만들어졌는지 속성을 추가해주는 것이다.

In [20]:
import numpy as np
if __name__ == '__main__':
  A = Square()
  B = Exp()
  C = Square()

  x = Variable(np.array(0.5))
  a = A(x)
  b = B(a)
  y = C(b)
  print(y.data)
  assert y.creator == C
  assert y.creator.input == b
  assert y.creator.input.creator == B
  assert y.creator.input.creator.input == a
  assert y.creator.input.creator.input.creator == A
  assert y.creator.input.creator.input.creator.input == x


1.648721270700128


### 7.2 역전파 도전

In [22]:
if __name__ == '__main__':
  y.grad = np.array(1.0)
  C = y.creator
  b = C.input
  b.grad = C.backward(y.grad)
  B = b.creator
  a = B.input
  a.grad = B.backward(b.grad)
  A = a.creator
  x = A.input
  x.grad = A.backward(a.grad)
  print(x.grad)

3.297442541400256


### 7.3 backward 메서드 추가

In [23]:
class Variable:
  def __init__(self, data) :
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func
  
  def backward(self):
    f = self.creator
    if f is not None:
      x = f.input
      x.grad = f.backward(self.grad)
      x.backward()

      

In [26]:
if __name__ == '__main__':
  A = Square()
  B = Exp()
  C = Square()

  x = Variable(np.array(0.5))
  a = A(x)
  b = B(a)
  y = C(b)

  y.grad = np.array(1.0)
  y.backward()
  print(x.grad)

3.297442541400256
